# Convert DHM Video Frames into GT Data
This converts each frame of the observed video data into a .pt file that can be loaded by Components/data_loader.FrameDataset.
These data files are primarly used during pose optimization as target amplitude and phase 

In [5]:
import os
import scipy
import torch
import matplotlib.pyplot as plt

def create_frame_file(idx, frame_data, path):
    
    opd_units = frame_data["opd_units"]
    px_size = frame_data["px_size"]

    
    amp = frame_data["amp"][idx]
    opd = frame_data["opd"][idx]
    
    amp = torch.from_numpy(amp).clone()
    opd = torch.from_numpy(opd).clone()
    
    # Guessed Pose: (fixed rotation axis with constant angular velocity over all frames)
    position = torch.tensor([25, 25, 25], dtype=torch.float64)
    offset = torch.tensor([0.0, 0.0, 0.0], dtype=torch.float64)
    rotation_axis = torch.tensor([1.0, 0.0, 0.0], dtype=torch.float64)
    angle = idx/max_frames * torch.tensor([360], dtype=torch.float64)
    
    
    
    data = {
        "frame_data_file": frame_data_file,
        "frame_idx": idx,
        
        "amp": amp,
        "opd": opd,
        "opd_units": opd_units[0],
        "px_size": px_size[0],
        
        "pose_unit": "um",
        "position": position,
        "offset": offset,
        "rotation_axis": rotation_axis,
        "angle": angle
    }
    
    
    torch.save(data, path)

In [6]:
# Copy from Create CSV.ipynb
import csv
def csv_from_list(data_paths, out_dir, file_name, mode="relative", custom_base=None):
    """
    Create a CSV listing files from a list of data paths.

    Modes:
    - 'relative': paths relative to the current working directory (e.g., notebook location)
    - 'filename': only the file names
    - 'absolute': full absolute paths
    - 'custom': use custom_base as the prefix path
    """
    out_file = os.path.join(out_dir, file_name)

    if mode == "custom" and not custom_base:
        raise ValueError("custom_base must be provided when mode='custom'")

    notebook_dir = os.getcwd()  # The directory where the notebook is running

    with open(out_file, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        for file_path in data_paths:
            if os.path.isfile(file_path):
                if mode == "relative":
                    path_to_write = os.path.relpath(file_path, start=notebook_dir)
                elif mode == "filename":
                    path_to_write = os.path.basename(file_path)
                elif mode == "absolute":
                    path_to_write = os.path.abspath(file_path)
                elif mode == "custom":
                    # Optional: support both str or Path object for custom_base
                    path_to_write = os.path.join(custom_base, os.path.basename(file_path))
                else:
                    raise ValueError(f"Unsupported mode: {mode}")

                writer.writerow([path_to_write])

In [7]:
# Frame Data source    
frame_data_path = "../../Data/HEK Cells/DHM/Raw Data/Frames/"
frame_data_file = "dhm_one_rev_centered_amp_opd_um.mat"

# GT Data Output
output_file_path = "../../Data/HEK Cells/DHM/GT Frames"
output_file_name = "DHM.pt"

# Load Data
frame_data_path = os.path.join(frame_data_path, frame_data_file)
frame_data = scipy.io.loadmat(frame_data_path)

# number of recorded frames
max_frames = frame_data["opd"].shape[0]

paths = []

# Number of created frames -> frame 0 to n_frames
n_frames = max_frames                               
for i in range(n_frames):
    
    file_name, file_extention = output_file_name.split(".")
    file_name = file_name + f"_{i:03d}." + file_extention
    path = os.path.join(output_file_path, file_name)

    create_frame_file(i, frame_data, path)
    
    paths.append(path)
    pass

# Create CSV file
csv_from_list(paths, output_file_path, "DHM_Frames.csv", mode="absolute")